In [1]:
# true if working from tbi, false is working from home
tbi = False

if tbi:
    base_folder = '/scr/aldea/kgutenbrunner/github/MA/'
else:
    base_folder = '/Users/katringutenbrunner/Desktop/MA/github/MA/'


In [2]:

import os
import csv
import re

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from Bio import AlignIO, SeqIO


import RNA

In [3]:
# get all the stk files except YFV
stk_folder = os.path.join(base_folder, 'data/mbfv_stk/')
stk_files = [os.path.join(stk_folder, file) for file in os.listdir(stk_folder) if file.endswith('xrRNA1.stk') and 'YFV' not in file]

In [4]:
# get the seqeuences from each stk file
sequences = []
rfs = []
for stk_file in stk_files:
    align = AlignIO.read(stk_file, "stockholm")
    rfs.append(align.column_annotations['reference_annotation'])
    for record in align:
        sequences.append(str(record.seq))



In [ ]:
output_fa_CM = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/data/combined.fa')
# create and fasta file from all the sequences
with open(output_fa_CM, 'w') as f:
    for stk_file in stk_files_CM:
        align = AlignIO.read(stk_file, "stockholm")
        for record in align:
            
            f.write(f'>{record.id}\n')
            f.write(f'{record.seq}\n')
            f.write(f"{align.column_annotations['secondary_structure']} #S\n")

In [ ]:
# use mlocarna to create a alignment file
!mlocarna $output_fa_CM

In [ ]:
# save the output consensus secondary structure from mlocarna
alifold = '...(((((((((.......)))).(((((.............)))))..)))))...............'

In [ ]:
# create a stockholm file based on the mlocarna alignment

# get the alignment file
input_file = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/data/combined.out/results/result.aln')
# output name for the stockholm file
align_stk_file = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/data/alignment.stk')

input_file = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/stk_files/combined.out/results/result.aln')


# Load the Clustal alignment
alignment = AlignIO.read(input_file, "clustal")
# add the secondary structure from mlocarna manually
alignment.column_annotations["secondary_structure_cons"] = alifold
alignment.column_annotations['secondary_structure'] = alifold

# Write the alignment in Stockholm format
AlignIO.write(alignment, align_stk_file, "stockholm")

In [ ]:
# create the covariance model based on the stockholm we just created
!cmbuild /scr/aldea/kgutenbrunner/working/cov_model/MBFV_xrRNA1/MBFV_xrRNA1.cm $align_stk_file

In [ ]:
# load the just create model
cm_model = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/MBFV_xrRNA1.cm')
designs_file_CM = os.path.join(base_folder, 'cov_model/MBFV_xrRNA1/data/designed_sequences.out')

In [ ]:
# create 100 sequences from the model
# !cmemit -N 100 -a -o $designs_file $cm_model